# AIhoonbot.com — 오프라인 Linux용 의존성 패키지 다운로더

### 사용 방법
1. 이 폴더(`colab-pack-deps`) 전체를 Google Drive에 업로드
2. 아래 셀을 **순서대로** 실행
3. 마지막 셀에서 `deps.tar.gz` 자동 다운로드
4. `deps.tar.gz`를 Linux 서버의 프로젝트 루트에 넣고:
   ```bash
   bash setup-linux.sh
   ```

## Step 1: Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Step 2: 경로 설정

`colab-pack-deps` 폴더가 Drive 어디에 있는지 확인 후 수정

In [ ]:
import os, shutil

PACK_DIR = '/content/drive/MyDrive/colab-pack-deps'

print('폴더 내용:')
for f in sorted(os.listdir(PACK_DIR)):
    print(' -', f)

## Step 3: 작업 디렉토리 초기화

In [ ]:
WORK = '/content/pack-work'
OUT  = '/content/deps-bundle'

shutil.rmtree(WORK, ignore_errors=True)
shutil.rmtree(OUT,  ignore_errors=True)
shutil.copytree(PACK_DIR, WORK)
os.makedirs(f'{OUT}/pip-packages', exist_ok=True)

print('준비 완료')

## Step 4: Python 패키지 다운로드 (Linux x86_64 .whl)

In [ ]:
!pip3 download \
    -r /content/pack-work/requirements-hoonbot.txt \
    -r /content/pack-work/requirements-claude.txt \
    -d /content/deps-bundle/pip-packages

n = len(os.listdir(f'{OUT}/pip-packages'))
print(f'\n[OK] Python 패키지 {n}개 다운로드 완료')

## Step 5: Node.js LTS 설치

In [ ]:
%%bash
curl -fsSL https://deb.nodesource.com/setup_lts.x | bash - > /dev/null 2>&1
apt-get install -y nodejs > /dev/null 2>&1
echo "node: $(node --version)"
echo "npm:  $(npm --version)"

## Step 6: npm install + node_modules 번들

> 이전 방식(npm cache)과 달리, 실제 `node_modules`를 tar로 번들합니다.  
> tar는 symlink를 보존하므로 npm workspace가 깨지지 않습니다.  
> 대상 서버에서 `npm rebuild`로 native 모듈(node-pty)만 재컴파일합니다.

In [ ]:
%%bash
cd /content/pack-work/Messenger
npm install
echo "[OK] npm install 완료"

In [ ]:
%%bash
cp -a /content/pack-work/Messenger/node_modules /content/deps-bundle/node_modules
echo "[OK] node_modules 복사 완료 ($(du -sh /content/deps-bundle/node_modules | cut -f1))"

## Step 7: node-gyp 헤더 캐시 (offline rebuild용)

In [ ]:
%%bash
npx node-gyp install 2>/dev/null || true
if [ -d "$HOME/.cache/node-gyp" ]; then
    cp -r "$HOME/.cache/node-gyp" /content/deps-bundle/node-gyp-cache
    echo "[OK] node-gyp 헤더 캐시됨"
elif [ -d "$HOME/.node-gyp" ]; then
    cp -r "$HOME/.node-gyp" /content/deps-bundle/node-gyp-cache
    echo "[OK] node-gyp 헤더 캐시됨"
else
    echo "[WARN] node-gyp 헤더를 찾을 수 없음"
fi

## Step 8: 버전 정보 기록

In [ ]:
%%bash
cat > /content/deps-bundle/versions.txt << EOF
node=$(node --version)
npm=$(npm --version)
python=$(python3 --version 2>&1 | awk '{print $2}')
EOF
echo "기록된 버전:"
cat /content/deps-bundle/versions.txt

## Step 9: deps.tar.gz 생성

In [ ]:
%%bash
cd /content
tar czf deps.tar.gz deps-bundle/
ls -lh deps.tar.gz
echo '[OK] deps.tar.gz 생성 완료'

## Step 10: 다운로드

In [ ]:
from google.colab import files
files.download('/content/deps.tar.gz')
print('다운로드 시작됨. 브라우저 다운로드 폴더 확인.')

---
## 다음 단계: Linux 서버에서 설치

```bash
# deps.tar.gz를 프로젝트 루트에 복사 후
cd ~/AIhoonbot.com
bash setup-linux.sh
# 설치 완료 후
bash start-all.sh
```

### 서버 사전 요구사항
- Python 3.x + pip3
- Node.js LTS (위 Step 5에서 설치된 것과 같은 메이저 버전)
- build-essential (setup-linux.sh가 자동 설치 시도)